In [13]:
#Set functions
def compute_histogram(image, bins=9):
    histogram, _ = np.histogram(image, bins=bins, range=(image.min(), image.max()))
    histogram = histogram / np.sum(histogram)
    return histogram

def extract_statistics(image):
    return {
        'Mean': round(np.mean(image), 3),
        'Std Dev': round(np.std(image), 3),
        'Max': round(np.max(image), 3),
        'Min': round(np.min(image), 3),
        'Median': round(np.median(image), 3)
    }


In [ ]:
#fft
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

def apply_fft(img):
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = np.log(np.abs(fshift) + 1)
    return f, fshift, magnitude_spectrum

def apply_ifft(fshift):
    f_ishift = np.fft.ifftshift(fshift)
    img_back = np.abs(np.fft.ifft2(f_ishift))
    return img_back

'''Loop 10 image from real and fake''' 
# Paths for real and fake image directories
real_image_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_real'  # Directory with real images
fake_image_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_fake'  # Directory with fake images
output_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\results\figures\fft'            # Directory to save processed images
# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
# Get list of images and limit to 10 each for processing
real_images = sorted([os.path.join(real_image_dir, f) for f in os.listdir(real_image_dir) if f.endswith('.jpg')])[:10]
fake_images = sorted([os.path.join(fake_image_dir, f) for f in os.listdir(fake_image_dir) if f.endswith('.jpg')])[:10]

'''Loop one image from real and fake''' 
# real_images = [r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_real\real_00002.jpg']
# fake_images = [r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_fake\easy_9_1010.jpg']

# Process 10 real and 10 fake images
bins = 256

for i, (real_image_path, fake_image_path) in enumerate(zip(real_images, fake_images)):
    # Load real and fake images as grayscale
    image_0 = cv2.imread(real_image_path, cv2.IMREAD_GRAYSCALE)
    image_1 = cv2.imread(fake_image_path, cv2.IMREAD_GRAYSCALE)
    # Apply FFT and IFFT for real image
    fft_0, fft_shift_0, fft_magnitude_0 = apply_fft(image_0)
    ifft_image_0 = apply_ifft(fft_shift_0)
    histogram_0 = compute_histogram(fft_magnitude_0, bins)
    stats_0 = extract_statistics(fft_magnitude_0)

    # Apply FFT and IFFT for fake image
    fft_1, fft_shift_1, fft_magnitude_1 = apply_fft(image_1)
    ifft_image_1 = apply_ifft(fft_shift_1)
    histogram_1 = compute_histogram(fft_magnitude_1, bins)
    stats_1 = extract_statistics(fft_magnitude_1)

    # Plot and save histograms and FFT results for each image
    fig, axs = plt.subplots(2, 3, figsize=(20, 6))

    # FFT plots in separate figure
    fig_fft, axs_fft = plt.subplots(2, 3, figsize=(20, 6))

    # Real Image
    axs_fft[0][0].imshow(image_0, cmap='gray')
    axs_fft[0][0].set_title('Original Real Image')
    axs_fft[0][0].axis('off')

    axs_fft[0][1].imshow(fft_magnitude_0, cmap='gray')
    axs_fft[0][1].set_title('FFT Magnitude Spectrum (Real)')
    axs_fft[0][1].axis('off')

    # Display the histogram of power spectrum as bar plot
    axs_fft[0][2].bar(range(bins), histogram_0, color='blue', alpha=0.7)
    axs_fft[0][2].set_title(f'Histogram (Real) - {bins} Bins')
    axs_fft[0][2].set_xlabel('Bins')
    axs_fft[0][2].set_ylabel('Frequency')

    # Fake Image
    axs_fft[1][0].imshow(image_1, cmap='gray')
    axs_fft[1][0].set_title('Original Fake Image')
    axs_fft[1][0].axis('off')

    axs_fft[1][1].imshow(fft_magnitude_1, cmap='gray')
    axs_fft[1][1].set_title('FFT Magnitude Spectrum (Fake)')
    axs_fft[1][1].axis('off')

    # Display the histogram of power spectrum as bar plot
    axs_fft[1][2].bar(range(bins), histogram_1, color='red', alpha=0.7)
    axs_fft[1][2].set_title(f'Histogram (Fake) - {bins} Bins')
    axs_fft[1][2].set_xlabel('Bins')
    axs_fft[1][2].set_ylabel('Frequency')

    # Display statistics below the images
    axs_fft[0][0].text(0.5, -0.2, f'Statistics (Real): {extract_statistics(fft_magnitude_0)}', 
                    transform=axs_fft[0][1].transAxes, fontsize=10, ha='center')
    axs_fft[1][0].text(0.5, -0.2, f'Statistics (Fake): {extract_statistics(fft_magnitude_1)}', 
                    transform=axs_fft[1][1].transAxes, fontsize=10, ha='center')

    plt.tight_layout()
    plt.show()



In [ ]:
#lbp
import os
from skimage.feature import local_binary_pattern
import numpy as np
import cv2
import matplotlib.pyplot as plt

def apply_lbp(img, radius=1, n_points=8):
    """
    Apply Local Binary Pattern (LBP) to detect texture inconsistencies in the images.
    
    Args:
        img (np.ndarray): Image to process.
        radius (int): Radius of the LBP pattern.
        n_points (int): Number of points considered in the LBP pattern.
        
    Returns:
        np.ndarray: LBP-processed image.
    """
    lbp = local_binary_pattern(img, n_points, radius, method='default')
    return lbp

def compute_histogram(image, bins):
    histogram, _ = np.histogram(image, bins=bins, density=True)
    return histogram

def extract_statistics(image):
    return {
        'Mean': round(np.mean(image), 3),
        'Std Dev': round(np.std(image), 3),
        'Max': round(np.max(image), 3),
        'Min': round(np.min(image), 3),
        'Median': round(np.median(image), 3)
    }

'''Loop 10 image from real and fake''' 
# Paths for real and fake image directories
real_image_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_real'  # Directory with real images
fake_image_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_fake'  # Directory with fake images
output_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\results\figures\lbp'            # Directory to save processed images
# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
# Get list of images and limit to 10 each for processing
real_images = sorted([os.path.join(real_image_dir, f) for f in os.listdir(real_image_dir) if f.endswith('.jpg')])[:10]
fake_images = sorted([os.path.join(fake_image_dir, f) for f in os.listdir(fake_image_dir) if f.endswith('.jpg')])[:10]

'''Loop one image from real and fake''' 
real_images = [r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_real\real_00002.jpg']
fake_images = [r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_fake\easy_9_1010.jpg']

for i, (real_image_path, fake_image_path) in enumerate(zip(real_images, fake_images)):
    # Load real and fake images as grayscale
    image_0 = cv2.imread(real_image_path, cv2.IMREAD_GRAYSCALE)
    image_1 = cv2.imread(fake_image_path, cv2.IMREAD_GRAYSCALE)

    # Apply LBP
    n_points = 8
    radius = 1
    lbp_0 = apply_lbp(image_0, radius, n_points)
    lbp_1 = apply_lbp(image_1, radius, n_points)

    # Compute histograms for LBP images
    bins = range(0, n_points + 3)
    bins = range(0, 256)
    lbp_hist_0 = compute_histogram(lbp_0, bins)
    lbp_hist_1 = compute_histogram(lbp_1, bins)

    # Set up the plots
    fig, axs = plt.subplots(2, 3, figsize=(15, 8))

    # Original Image (Real)
    axs[0, 0].imshow(image_0, cmap='gray')
    axs[0, 0].set_title('Original Real Image')
    axs[0, 0].axis('off')

    # LBP Image (Real)
    axs[0, 1].imshow(lbp_0, cmap='gray')
    axs[0, 1].set_title('LBP Image (Real)')
    axs[0, 1].axis('off')
    axs[0, 1].text(0.5, -0.2, f'Statistics: {extract_statistics(lbp_0)}', transform=axs[0, 1].transAxes, fontsize=12, ha='center')

    # Histogram of LBP (Real)
    axs[0, 2].bar(bins[:-1], lbp_hist_0, width=0.5, color='gray', alpha=0.7)
    axs[0, 2].set_title('LBP Histogram (Real)')
    axs[0, 2].set_xlabel('LBP Value')
    axs[0, 2].set_ylabel('Normalized Frequency')

    # Original Image (Fake)
    axs[1, 0].imshow(image_1, cmap='gray')
    axs[1, 0].set_title('Original Fake Image')
    axs[1, 0].axis('off')

    # LBP Image (Fake)
    axs[1, 1].imshow(lbp_1, cmap='gray')
    axs[1, 1].set_title('LBP Image (Fake)')
    axs[1, 1].axis('off')
    axs[1, 1].text(0.5, -0.2, f'Statistics: {extract_statistics(lbp_1)}', transform=axs[1, 1].transAxes, fontsize=12, ha='center')

    # Histogram of LBP (Fake)
    axs[1, 2].bar(bins[:-1], lbp_hist_1, width=0.5, color='gray', alpha=0.7)
    axs[1, 2].set_title('LBP Histogram (Fake)')
    axs[1, 2].set_xlabel('LBP Value')
    axs[1, 2].set_ylabel('Normalized Frequency')

    # Save the figure
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'lbp_comparison_{i+1}.png'))
    plt.close(fig)


In [ ]:
#sobel
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt


bins=256

'''Loop 10 image from real and fake''' 
# Paths for real and fake image directories
real_image_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_real'
fake_image_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_fake'
output_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\results\figures\sobel'
# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
# Get list of images and limit to 10 each for processing
real_images = sorted([os.path.join(real_image_dir, f) for f in os.listdir(real_image_dir) if f.endswith('.jpg')])[:10]
fake_images = sorted([os.path.join(fake_image_dir, f) for f in os.listdir(fake_image_dir) if f.endswith('.jpg')])[:10]

'''Loop one image from real and fake''' 
real_images = [r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_real\real_00002.jpg']
fake_images = [r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_fake\easy_9_1010.jpg']

# Process each real and fake image
for i, (real_image_path, fake_image_path) in enumerate(zip(real_images, fake_images)):
    # Load real and fake images as grayscale
    image_real = cv2.imread(real_image_path, cv2.IMREAD_GRAYSCALE)
    image_fake = cv2.imread(fake_image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply Sobel filter for real image
    sobel_x_real = cv2.Sobel(image_real, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y_real = cv2.Sobel(image_real, cv2.CV_64F, 0, 1, ksize=3)
    sobel_magnitude_real = np.sqrt(sobel_x_real**2 + sobel_y_real**2)
    
    # Calculate histogram for Sobel magnitude (real)
    hist_real, bin_edges_real = np.histogram(sobel_magnitude_real, bins=bins)
    hist_real = hist_real / np.sum(hist_real)  # Normalize histogram
    
    # Apply Sobel filter for fake image
    sobel_x_fake = cv2.Sobel(image_fake, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y_fake = cv2.Sobel(image_fake, cv2.CV_64F, 0, 1, ksize=3)
    sobel_magnitude_fake = np.sqrt(sobel_x_fake**2 + sobel_y_fake**2)
    
    # Calculate histogram for Sobel magnitude (fake)
    hist_fake, bin_edges_fake = np.histogram(sobel_magnitude_fake, bins=bins)
    hist_fake = hist_fake / np.sum(hist_fake)  # Normalize histogram

    # Plot and save histograms
    fig, axs = plt.subplots(2, 3, figsize=(15, 8))
    
    # Original Image (Real)
    axs[0, 0].imshow(image_real, cmap='gray')
    axs[0, 0].set_title('Original Real Image')
    axs[0, 0].axis('off')

    axs[0, 1].imshow(sobel_magnitude_real, cmap='gray')
    axs[0, 1].set_title('Sobel Image (Real)')
    axs[0, 1].axis('off')
    axs[0, 1].text(0.5, -0.2, f'Statistics: {extract_statistics(sobel_magnitude_real)}', transform=axs[0, 1].transAxes, fontsize=12, ha='center')

    # Histogram for real image
    axs[0,2].plot(hist_real, color='blue')
    axs[0,2].set_title(f"Sobel Magnitude Histogram (Real Image {i+1})")
    axs[0,2].set_xlabel("Gradient Magnitude")
    axs[0,2].set_ylabel("Frequency")
    
    # Histogram for fake image
    # Original Image (Real)
    axs[1, 0].imshow(image_fake, cmap='gray')
    axs[1, 0].set_title('Original Fake Image')
    axs[1, 0].axis('off')

    axs[1, 1].imshow(sobel_magnitude_fake, cmap='gray')
    axs[1, 1].set_title('Sobel Image (Fake)')
    axs[1, 1].axis('off')
    axs[1, 1].text(0.5, -0.2, f'Statistics: {extract_statistics(sobel_magnitude_fake)}', transform=axs[1, 1].transAxes, fontsize=12, ha='center')

    axs[1,2].plot(hist_fake, color='red')
    axs[1,2].set_title(f"Sobel Magnitude Histogram (Fake Image {i+1})")
    axs[1,2].set_xlabel("Gradient Magnitude")
    axs[1,2].set_ylabel("Frequency")
    
    # Save the histogram plot
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'sobel_histogram_{i+1}.png'))
    plt.close(fig)

    # Save Sobel magnitude images for visualization if needed
    # cv2.imwrite(os.path.join(output_dir, f'real_sobel_{i+1}.jpg'), sobel_magnitude_real)
    # cv2.imwrite(os.path.join(output_dir, f'fake_sobel_{i+1}.jpg'), sobel_magnitude_fake)

    # Print histogram values for debugging
    print(f"Histogram for Real Image {i+1}: {hist_real}")
    print(f"Histogram for Fake Image {i+1}: {hist_fake}")


Histogram for Real Image 1: [2.87363889e-01 6.31333333e-02 6.15277778e-02 6.32277778e-02
 5.68500000e-02 4.54972222e-02 3.51500000e-02 3.07611111e-02
 2.64805556e-02 2.01916667e-02 1.84194444e-02 1.74611111e-02
 1.45750000e-02 1.32666667e-02 1.21250000e-02 1.16138889e-02
 1.04250000e-02 9.83333333e-03 9.81388889e-03 8.66111111e-03
 8.55555556e-03 7.52500000e-03 7.90833333e-03 6.87222222e-03
 7.00277778e-03 6.29166667e-03 5.92777778e-03 5.66111111e-03
 5.27222222e-03 4.83888889e-03 5.05277778e-03 4.59444444e-03
 4.08333333e-03 4.16388889e-03 4.07500000e-03 3.89166667e-03
 3.55833333e-03 3.48888889e-03 3.13888889e-03 3.18888889e-03
 2.83333333e-03 3.01944444e-03 2.74166667e-03 2.70833333e-03
 2.46111111e-03 2.62222222e-03 2.37777778e-03 2.22777778e-03
 2.23333333e-03 2.05833333e-03 1.98333333e-03 2.00833333e-03
 1.83611111e-03 1.81666667e-03 1.85833333e-03 1.57222222e-03
 1.56666667e-03 1.42500000e-03 1.47777778e-03 1.43611111e-03
 1.50555556e-03 1.24722222e-03 1.05000000e-03 1.16666667e

In [ ]:
#clahe
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Define the CLAHE function
def apply_clahe(img, clip_limit=2.0, tile_grid_size=(8, 8)):
    """
    Apply Contrast Limited Adaptive Histogram Equalization (CLAHE) to an image.

    Parameters:
    - image: Input image (grayscale or color).
    - clip_limit: Threshold for contrast limiting.
    - tile_grid_size: Size of grid for histogram equalization (height, width).

    Returns:
    - clahe_image: Image after applying CLAHE.
    """
    # Create a CLAHE object
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)

    # Apply CLAHE to the grayscale image
    clahe_image = clahe.apply(img)

    return clahe_image

bins=256

'''Loop 10 image from real and fake''' 
# Paths for real and fake image directories
real_image_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_real'
fake_image_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_fake'
output_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\results\figures\clahe'
# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
# Get list of images and limit to 10 each for processing
real_images = sorted([os.path.join(real_image_dir, f) for f in os.listdir(real_image_dir) if f.endswith('.jpg')])[:10]
fake_images = sorted([os.path.join(fake_image_dir, f) for f in os.listdir(fake_image_dir) if f.endswith('.jpg')])[:10]

'''Loop one image from real and fake''' 
real_images = [r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_real\real_00002.jpg']
fake_images = [r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_fake\easy_9_1010.jpg']

for i, (real_image_path, fake_image_path) in enumerate(zip(real_images, fake_images)):
    # Load real and fake images as grayscale
    image_real = cv2.imread(real_image_path, cv2.IMREAD_GRAYSCALE)
    image_fake = cv2.imread(fake_image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply CLAHE
    clahe_image_0 = apply_clahe(image_real)
    clahe_image_1 = apply_clahe(image_fake)

    # Compute histograms
    bins=256
    
    hist_0=compute_histogram(clahe_image_0, bins=bins)
    hist_1=compute_histogram(clahe_image_1, bins=bins)
    # hist_0 = cv2.calcHist([clahe_image_0], [0], None, [bins], [0, bins])
    # hist_1 = cv2.calcHist([clahe_image_1], [0], None, [bins], [0, bins])

    # Create plots
    fig_clahe, axs_clahe = plt.subplots(2, 3, figsize=(18, 10))

    # Original Real Image
    axs_clahe[0, 0].imshow(image_real, cmap='gray')  # Convert BGR to RGB for correct color representation
    axs_clahe[0, 0].set_title('Original Real Image')
    axs_clahe[0, 0].axis('off')

    # CLAHE Image for Real
    axs_clahe[0, 1].imshow(clahe_image_0, cmap='gray')
    axs_clahe[0, 1].set_title('CLAHE (Real Image)')
    axs_clahe[0, 1].axis('off')

    # Histogram for CLAHE Real
    axs_clahe[0, 2].plot(hist_0)
    axs_clahe[0, 2].set_title('Histogram (CLAHE Real Image)')
    axs_clahe[0, 2].set_xlim([0, bins])

    # Original Fake Image
    axs_clahe[1, 0].imshow(image_fake, cmap='gray')  # Convert BGR to RGB for correct color representation
    axs_clahe[1, 0].set_title('Original Fake Image')
    axs_clahe[1, 0].axis('off')

    # CLAHE Image for Fake
    axs_clahe[1, 1].imshow(clahe_image_1, cmap='gray')
    axs_clahe[1, 1].set_title('CLAHE (Fake Image)')
    axs_clahe[1, 1].axis('off')

    # Histogram for CLAHE Fake
    axs_clahe[1, 2].plot(hist_1)
    axs_clahe[1, 2].set_title('Histogram (CLAHE Fake Image)')
    axs_clahe[1, 2].set_xlim([0, bins])

    # Display statistics
    axs_clahe[0, 0].text(1, -0.3, f'Statistics (Real): {extract_statistics(clahe_image_0)}', 
                        transform=axs_clahe[0, 0].transAxes, fontsize=12, ha='center')

    axs_clahe[1, 0].text(1, -0.3, f'Statistics (Fake): {extract_statistics(clahe_image_1)}', 
                        transform=axs_clahe[1, 0].transAxes, fontsize=12, ha='center')

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'CLAHE_histogram_{i+1}.png'))
    plt.close(fig_clahe)

